# **Vertex AI SDK: AutoML tabular forecasting model for batch prediction**

## 🔧 **Setting up**



### 1. Open the notebook

[Colab Enterprise](https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Froitraining%2Fgcp-demos%2Frefs%2Fheads%2Fmaster%2Fai%2Fautoml%2Fautoml_forecasting.ipynb)

### 2. Install packages, set values, create a bucket


In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform google-cloud-bigquery

In [ ]:
# Initialize constants and variables
# replace <project-id> with your project info
PROJECT_ID = "<project-id>"
LOCATION = "us-central1"
BUCKET_URI = f"gs://{PROJECT_ID}-automl"
MODEL_DISPLAY_NAME = "iowa-liquor-sales-forecast-model"

time_column = "date"
time_series_identifier_column = "store_name"
target_column = "sale_dollars"

COLUMN_SPECS = {
    time_column: "timestamp",
    target_column: "numeric",
    "city": "categorical",
    "zip_code": "categorical",
    "county": "categorical",
}

In [ ]:
# set the default project for Google Cloud SDK
! gcloud config set project {PROJECT_ID}

# Create the Cloud Storage bucket
# You can ignore the error reported if the bucket already exists
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

In [ ]:
# import libraries and initialize connection to aiplatform
import urllib

from google.cloud import aiplatform, aiplatform_v1, bigquery
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

In [ ]:
# set up for BigQuery actions
batch_predict_bq_output_dataset_name = "iowa_liquor_sales_predictions"
batch_predict_bq_output_dataset_path = "{}.{}".format(
PROJECT_ID, batch_predict_bq_output_dataset_name
)

batch_predict_bq_output_uri_prefix = "bq://{}.{}".format(
    PROJECT_ID, batch_predict_bq_output_dataset_name
)

TRAINING_DATASET_BQ_PATH = (
    "bq://bigquery-public-data:iowa_liquor_sales_forecasting.2020_sales_train"
)

PREDICTION_DATASET_BQ_PATH = (
    "bq://bigquery-public-data:iowa_liquor_sales_forecasting.2021_sales_predict"
)

client = bigquery.Client(project=PROJECT_ID)

In [ ]:
# create the dataset
# skip this if you plan on using existing data from an earlier run
bq_dataset_id = bigquery.Dataset(batch_predict_bq_output_dataset_path)
dataset_location = "US"
bq_dataset_id.location = dataset_location


try:
    client.delete_dataset(bq_dataset_id, delete_contents=True)
except Exception as e:
    print(e)
bq_dataset = client.create_dataset(bq_dataset_id)
print(
    "Created bigquery dataset {} in {}".format(
        batch_predict_bq_output_dataset_path, dataset_location
    )
)

## **Creating the model**

Use this section if you haven't already trained a model. It will take a couple hours to complete the training.

In [ ]:
# create the vertex AI dataset from the data in bigquery
dataset = aiplatform.TimeSeriesDataset.create(
    display_name="iowa_liquor_sales_train",
    bq_source=[TRAINING_DATASET_BQ_PATH],
)

print(dataset.resource_name)

In [ ]:
# create the automl training job object
training_job = aiplatform.AutoMLForecastingTrainingJob(
    display_name=MODEL_DISPLAY_NAME,
    optimization_objective="minimize-rmse",
    column_specs=COLUMN_SPECS,
)

In [ ]:
# run the training job and outpput the model
model = training_job.run(
    dataset=dataset,
    target_column=target_column,
    time_column=time_column,
    time_series_identifier_column=time_series_identifier_column,
    available_at_forecast_columns=[time_column],
    unavailable_at_forecast_columns=[target_column],
    time_series_attribute_columns=["city", "zip_code", "county"],
    forecast_horizon=30,
    context_window=30,
    data_granularity_unit="day",
    data_granularity_count=1,
    weight_column=None,
    budget_milli_node_hours=1000,
    model_display_name=MODEL_DISPLAY_NAME,
    predefined_split_column_name=None,
)

## **Using an existing model**

Use this if you've already trained the model per above, and have the model stored in your registry

In [ ]:
# use the python SDK to create a model object representing the model in the
# registry
parent = f"projects/{PROJECT_ID}/locations/{LOCATION}"
api_endpoint = f"{LOCATION}-aiplatform.googleapis.com"

model_client = aiplatform_v1.ModelServiceClient(
    client_options={"api_endpoint": api_endpoint}
)

target_model = None
for model in model_client.list_models(request={"parent": parent}):
    if model.display_name == MODEL_DISPLAY_NAME:
        target_model = model
        break
if not target_model:
    raise RuntimeError(
        f"No model found with display_name '{MODEL_DISPLAY_NAME}' in {parent}"
    )

model = aiplatform.Model(model_name=target_model.name)

## **Making predictions**

In [ ]:
# create a batch prediction job with some inputs from bigquery
batch_prediction_job = model.batch_predict(
    job_display_name="iowa_liquor_sales_forecasting_predictions",
    bigquery_source=PREDICTION_DATASET_BQ_PATH,
    instances_format="bigquery",
    bigquery_destination_prefix=batch_predict_bq_output_uri_prefix,
    predictions_format="bigquery",
    generate_explanation=True,
    sync=False,
)

print(batch_prediction_job)

In [ ]:
# wait for the job to complete. This can take 25+ minutes to complete
batch_prediction_job.wait()

In [ ]:
# show the results
for row in batch_prediction_job.iter_outputs():
    print(row)

## **Creating a dashboard**

Lastly, follow the given link to visualize the generated forecasts in [Data Studio](https://support.google.com/datastudio/answer/6283323?hl=en).

In [ ]:
tables = client.list_tables(batch_predict_bq_output_dataset_path)

prediction_table_id = ""
for table in tables:
    if (
        table.table_id.startswith("predictions_")
        and table.table_id > prediction_table_id
    ):
        prediction_table_id = table.table_id
batch_predict_bq_output_uri = "{}.{}".format(
    batch_predict_bq_output_dataset_path, prediction_table_id
)


def _sanitize_bq_uri(bq_uri):
    if bq_uri.startswith("bq://"):
        bq_uri = bq_uri[5:]
    return bq_uri.replace(":", ".")


def get_data_studio_link(
    batch_prediction_bq_input_uri,
    batch_prediction_bq_output_uri,
    time_column,
    time_series_identifier_column,
    target_column,
):
    batch_prediction_bq_input_uri = _sanitize_bq_uri(batch_prediction_bq_input_uri)
    batch_prediction_bq_output_uri = _sanitize_bq_uri(batch_prediction_bq_output_uri)
    base_url = "https://datastudio.google.com/c/u/0/reporting"
    query = (
        "SELECT \\n"
        " CAST(input.{} as DATETIME) timestamp_col,\\n"
        " CAST(input.{} as STRING) time_series_identifier_col,\\n"
        " CAST(input.{} as NUMERIC) historical_values,\\n"
        " CAST(predicted_{}.value as NUMERIC) predicted_values,\\n"
        " * \\n"
        "FROM `{}` input\\n"
        "LEFT JOIN `{}` output\\n"
        "ON\\n"
        "CAST(input.{} as DATETIME) = CAST(output.{} as DATETIME)\\n"
        "AND CAST(input.{} as STRING) = CAST(output.{} as STRING)"
    )
    query = query.format(
        time_column,
        time_series_identifier_column,
        target_column,
        target_column,
        batch_prediction_bq_input_uri,
        batch_prediction_bq_output_uri,
        time_column,
        time_column,
        time_series_identifier_column,
        time_series_identifier_column,
    )
    params = {
        "templateId": "067f70d2-8cd6-4a4c-a099-292acd1053e8",
        "ds0.connector": "BIG_QUERY",
        "ds0.projectId": PROJECT_ID,
        "ds0.billingProjectId": PROJECT_ID,
        "ds0.type": "CUSTOM_QUERY",
        "ds0.sql": query,
    }
    params_str_parts = []
    for k, v in params.items():
        params_str_parts.append('"{}":"{}"'.format(k, v))
    params_str = "".join(["{", ",".join(params_str_parts), "}"])
    return "{}?{}".format(base_url, urllib.parse.urlencode({"params": params_str}))


print(
    get_data_studio_link(
        PREDICTION_DATASET_BQ_PATH,
        batch_predict_bq_output_uri,
        time_column,
        time_series_identifier_column,
        target_column,
    )
)

## **Cleaning up**

In [ ]:
# Delete dataset
dataset.delete()

# Training job
training_job.delete()

# Delete model
model.delete()

# Delete batch prediction job
batch_prediction_job.delete()

# Delete the dataset
try:
    client.delete_dataset(bq_dataset_id, delete_contents=True, not_found_ok=True)
except Exception as e:
    print(e)

# Set this to true only if you'd like to delete your bucket
delete_bucket = False  # set True for deletion

if delete_bucket:
    ! gsutil rm -r $BUCKET_URI